# 语言模型

*语言模型*（language model）的目标是估计文本序列的联合概率，
是自然语言处理的关键。

长度为$T$的文本序列的联合概率
$P(x_1, x_2, \ldots, x_T)$，
其中 $x_t$（$1 \leq t \leq T$）文本序列在时间步$t$的词元（单词或字符）。

文本序列的联合概率，
可以转化为条件概率
$P(x_1, x_2, \ldots, x_T) = \prod_{t=1}^T P(x_t  \mid  x_1, \ldots, x_{t-1})$。
其中单词的概率以及给定前面几个单词后出现某个单词的条件概率，被称为*语言模型的参数*。


例如：
$
P(\text{deep}, \text{learning}, \text{is}, \text{fun}) =  P(\text{deep}) P(\text{learning}  \mid  \text{deep}) P(\text{is}  \mid  \text{deep}, \text{learning}) P(\text{fun}  \mid  \text{deep}, \text{learning}, \text{is})
$


$
\\
$
语言模型可以用于语音识别消歧，
比如“recognize speech”和“wreck a nice beach”读音相似，
很容易通过语言模型来解决。


## $n$-gram

深度学习的解决方案之前，
$n$-gram是最实用的语言模型。
假设序列满足$n$阶马尔可夫性质，
分别对应$n$元语法。


以下分别是“一元语法”（unigram）、“二元语法”（bigram）和“三元语法”（trigram）模型。
$$
\begin{aligned}
P(x_1, x_2, \ldots, x_t) &=  P(x_1) P(x_2) \ldots P(x_t),\\
P(x_1, x_2, \ldots, x_t) &=  P(x_1) P(x_2  \mid  x_1) \ldots P(x_t  \mid  x_t-1),\\
P(x_1, x_2, \ldots, x_t) &=  P(x_1) P(x_2  \mid  x_1) P(x_3  \mid  x_1, x_2) \ldots P(x_t  \mid  x_t-2, x_t-1).
\end{aligned}
$$

### $n$-gram主要问题

- $n$-gram 词频受*齐普夫定律*支配会迅速衰减。解决方案是*拉普拉斯平滑*。
- $n$-gram 没有考虑词的语义相似性，比如：cat和feline（猫科动物）可能出现在相关的上下文中。解决方案是*深度学习解决方案*。


### 拉普拉斯平滑

拉普拉斯平滑法可以有效地处理结构丰富而频率不足的低频词词组。

$$
\begin{aligned}
    \hat{P}(x) & = \frac{n(x) + \epsilon_1/m}{n + \epsilon_1}, \\
    \hat{P}(x' \mid x) & = \frac{n(x, x') + \epsilon_2 \hat{P}(x')}{n(x) + \epsilon_2}, \\
    \hat{P}(x'' \mid x,x') & = \frac{n(x, x',x'') + \epsilon_3 \hat{P}(x'')}{n(x, x') + \epsilon_3}.
\end{aligned}
$$
其中$n$表示训练集中的单词总数，用$m$表示唯一单词的数量。
$\epsilon_1,\epsilon_2$和$\epsilon_3$是超参数。
以$\epsilon_1$为例：当$\epsilon_1 = 0$时，不应用平滑；
当$\epsilon_1$接近正无穷大时，$\hat{P}(x)$接近均匀概率分布$1/m$。


### 齐普夫定律

*齐普夫定律*（Zipf's law）是指词频以一种明确的方式迅速衰减。
<!-- ![齐普夫定律](img/zipf's_law.png) -->
<!-- <img src="img/zipf's_law.png" width="350" height="350"  align=left/> -->
<img src="img/zipf's_law.png" width="40%" height="40%" align="left"/>

消除前几个例外单词后，剩余的所有单词大致遵循双对数坐标图上的一条直线。
即第$i$个最常用单词的频率$\log n_i = -\alpha \log i + c$其中$\alpha$是刻画分布的指数，$c$是常数。

齐普夫定律揭示通过计数统计和平滑来建模单词是*不可行*的，
因为这样建模的结果会大大*高估尾部单词的频率*。

## 循环神经网络

$n$元语法模型单词$x_t$在时间步$t$的条件概率仅取决于前面$n-1$个单词。
对于时间步$t-(n-1)$之前的单词，
如果我们想将其可能产生的影响合并到$x_t$上，
需要增加$n$，然而模型参数的数量也会随之呈指数增长，
因为词表$\mathcal{V}$需要存储$|\mathcal{V}|^n$个数字。

使用隐变量模型 $P(x_t \mid x_{t-1}, \ldots, x_1) \approx P(x_t \mid h_{t-1})$
其中$h_{t-1}$是*隐状态*（hidden state），
它存储了到时间步$t-1$的序列信息。

时间步$t$处的隐状态由当前输入$x_{t}$和先前隐状态$h_{t-1}$共同决定$h_t = f(x_{t}, h_{t-1})$。

### 有隐状态的循环神经网络

对隐状态使用循环计算的神经网络称为*循环神经网络*（recurrent neural network, RNN），
循环神经网络的隐状态可以捕获直到当前时间步序列的历史信息。

循环神经网络由*循环层*（recurrent layer）和输出层两部分构成，
*循环*是指计算循环，模型参数是固定的，不会随着时间步的增加而增加。

<img src="img/rnn.svg" width="50%" height="50%" align="left"/>

$$
\begin{aligned}
\mathbf{H}_t &= \phi(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}  + \mathbf{b}_h)\\
\mathbf{O}_t &= \mathbf{H}_t \mathbf{W}_{hq} + \mathbf{b}_q
\end{aligned}
$$

其中
$\mathbf{X}_t \in \mathbb{R}^{n \times d}$，
$\mathbf{H}_{t-1} \in \mathbb{R}^{n \times h}$，
$\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}$，
$\mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$，
$\mathbf{b}_h \in \mathbb{R}^{1 \times h}$，
$\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$，
$\mathbf{b}_q \in \mathbb{R}^{1 \times q}$
$\longrightarrow$
$\mathbf{H}_t \in \mathbb{R}^{n \times h}$，
$\mathbf{O}_t \in \mathbb{R}^{n \times q}$。

### 基于循环神经网络的字符级语言模型

语言模型的目标是根据过去的和当前的词元预测下一个词元，
因此我们将原始序列移位一个词元作为标签。
Bengio等人首先提出使用神经网络进行语言建模
 :cite:`Bengio.Ducharme.Vincent.ea.2003`。
 
下图是使用循环神经网络构建字符级语言模型。

<img src="img/rnn-train.svg" width="40%" height="40%" align="left"/>

## 困惑度（Perplexity）

可以通过一个序列中所有的$n$个词元的交叉熵损失的平均值来衡量：

$$\frac{1}{n} \sum_{t=1}^n -\log P(x_t \mid x_{t-1}, \ldots, x_1),$$

其中$P$由语言模型给出，
$x_t$是在时间步$t$从该序列中观察到的实际词元。
这使得不同长度的文档的性能具有了可比性。


由于历史原因，自然语言处理的科学家更喜欢使用*困惑度*（perplexity）。

$$\exp\left(-\frac{1}{n} \sum_{t=1}^n \log P(x_t \mid x_{t-1}, \ldots, x_1)\right).$$

困惑度的最好的理解是“下一个词元的实际选择数的调和平均数”。

* 在最好的情况下，模型总是完美地估计标签词元的概率为1。
  在这种情况下，模型的困惑度为1。
* 在最坏的情况下，模型总是预测标签词元的概率为0。
  在这种情况下，困惑度是正无穷大。
* 在基线上，该模型的预测是词表的所有可用词元上的均匀分布。
  在这种情况下，困惑度等于词表中唯一词元的数量。
  这是任何实际模型都必须超越这个上限。